In [18]:
from datetime import datetime, timedelta
import requests


In [12]:
url_countries = f"https://apiv3.apifootball.com/?action=get_countries&APIkey={API_TOKEN}"
COUNTRY_ID    = 27 #Brasil

In [13]:
url_competicoes = f"https://apiv3.apifootball.com/?action=get_leagues&country_id={COUNTRY_ID}&APIkey={API_TOKEN}"
LEAGUE_ID = 99 #Serie A


In [14]:
INICIO = '2024-07-01'
FIM    = '2024-08-01'
url_event = f'https://apiv3.apifootball.com/?action=get_events&from={INICIO}&to={FIM}&league_id={LEAGUE_ID}&APIkey={API_TOKEN}'

In [15]:
def get_stats(match_id):
    url_stats = f'https://apiv3.apifootball.com/?action=get_statistics&match_id={match_id}&APIkey={API_TOKEN}'
    response  = requests.get(url_stats)
    return response

def get_odds(match_id):
    url_odds  = f'https://apiv3.apifootball.com/?action=get_odds&from={INICIO}&to={FIM}&match_id={match_id}&APIkey={API_TOKEN}'
    response  = requests.get(url_odds)
    return response

def get_head2head(home_team_id, away_team_id):
    url_head2head = f"https://apiv3.apifootball.com/?action=get_H2H&firstTeamId={home_team_id}&secondTeamId={away_team_id}&APIkey={API_TOKEN}"
    response  = requests.get(url_head2head)
    return response

def get_predictions(match_id):
    url_predictions = f"https://apiv3.apifootball.com/?action=get_predictions&from={INICIO}&to={FIM}&match_id={match_id}&APIkey={API_TOKEN}"
    response  = requests.get(url_predictions)
    return response


def create_slug(key, data):
    key           = key.replace('/','-')
    completion    = f"{data['league_year']}-{data['match_hometeam_name']}-{data['match_awayteam_name']}"
    complete_path = slugify(key.replace('z','s') +'-'+completion)
    return complete_path 

In [16]:
def fix_timezone(data):
    data_completa = data['match_date'] + ' ' + data['match_time']
    date_format   = '%Y-%m-%d %H:%M'
    date_obj      = datetime.strptime(data_completa, date_format)
    new_date      = date_obj - timedelta(hours=5)
    str_date      = new_date.strftime(date_format)
    match_date    = str_date.split()[0]
    match_time    = str_date.split()[1]
    data['datetime_match'] = str_date
    data['match_date']     = match_date
    data['match_time']     = match_time
    return data
    
    

In [20]:
response     = requests.get(url_event)
matches      = response.json()
print(matches)
next_matches = [x for x in matches if x['match_status'] not in ['Finished', 'Postponed']]

{'error': 404, 'message': 'Authentification failed!'}


TypeError: string indices must be integers

In [7]:
data_cache = {}
for match in next_matches:
    country   = slugify(match['country_name'])
    league    = slugify(match['league_name'])
    path_data = f'{country}/{league}'
    match     = fix_timezone(match)
    match['url_match'] = create_slug(path_data, match)
    if match['match_date'] in data_cache:
        if path_data in data_cache[match['match_date']]:
            data_cache[match['match_date']][path_data].append(match)
        else:
            data_cache[match['match_date']].update({path_data : [match]})
    else:
        data_cache[match['match_date']] = {path_data : [match]}

NameError: name 'next_matches' is not defined

In [8]:
redisdb = RedisConnector()

TypeError: __init__() missing 4 required positional arguments: 'host', 'port', 'channels', and 'service_name'

In [11]:
for data, competicoes in data_cache.items():
    for competicao, matches in competicoes.items():
        complete_path = f'{competicao}/matches/{data}'
        redisdb.set(complete_path, matches)

In [12]:
all_data  = redisdb.all('/')[0]
dict_data = {}
for key, value in all_data.items():
    chave   = key.decode("utf-8")
    momment = key.decode("utf-8").split('/')[-1]
    if momment not in data_cache.keys() and 'macthes' in chave:
        redisdb.delete(chave)

In [13]:
for data, competicoes in data_cache.items():
    for competicao, matches in competicoes.items():
        for match in matches:
            stats = get_stats(match['match_id'])
            match['stats'] = stats.json()
            
            odds  = get_odds(match['match_id'])
            match['odds'] = odds.json()
            
            h2h   = get_head2head(match['match_hometeam_id'], match['match_awayteam_id'])
            match['h2h'] = h2h.json()
            
            pred  = get_predictions(match['match_id'])
            match['pred'] = pred.json()
            
            slug_path     = create_slug(competicao, match)
            complete_path = f"{competicao}/{slug_path}"
            redisdb.set(complete_path, match)